# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is the SysML v2 Kerbal model written by Bjorn Cole

The textual SysML v2 model is the Kerbal model:

    package Kerbal {
        package 'Rocket Building' {
            import ScalarFunctions::*;

            part def Rocket {
                part stages : 'Rocket Stage' [1..5] {
                    // placing this here because previous stages only make sense in context of a full vehicle
                    ref 'Carried Stage' : 'Rocket Stage' [1];
                    attribute 'Payload Mass' : Real;
                    attribute 'Loaded Mass' : Real;
                    attribute 'Burnout Mass' : Real;
                    part 'Coupler to Carrying Stage' : Coupler [0..8] {
                        attribute 'Separation Force' : Real;
                    }
                }
            }
            item def Oxidizer;
            item def Fuel;
            item def 'Solid Propellant';

            // TODO: Something something part symmetry
            abstract part def 'Rocket Stage';

            // use stage types to enforce matching
            part def 'Liquid Stage' :> 'Rocket Stage' {
                part engines : 'Liquid Engine' [0..8];
                part tanks : 'Fuel Tank Section' [0..30];
                attribute 'Full Mass' : Real = sum(engines->collect p:'Kerbal Rocket Part' (p::Mass)) +
                                        sum(tanks->collect p:'Fuel Tank Section' (p::'Full Mass'));

                attribute 'Empty Mass' : Real;
            }
            part def 'Solid Stage' :> 'Rocket Stage' {
                part boosters : 'Solid Booster' [0..8];
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            part def 'Coupler' :> 'Kerbal Rocket Part';

            abstract part def 'Fuel Tank Section' :> 'Kerbal Rocket Part' {
                attribute 'Liquid Fuel' : Real;
                attribute 'Oxidizer' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            abstract part def 'Liquid Engine' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Thrust' : Real;
            }
            abstract part def 'Solid Booster' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
                attribute 'Thrust' : Real;
            }

            part def 'Pod' :> 'Kerbal Rocket Part'  {
                attribute Torque : Real;
            }
            part def 'Parachute' :> 'Kerbal Rocket Part';

            part def 'Kerbal Rocket Part' {
                attribute Mass : Real;
                attribute 'Max Temperature' : Real;
            }
        }
        package 'Parts Library' {
            import ScalarFunctions::*;
            part def 'FL-T200 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 1.125;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.125;
            }
            part def 'FL-T100 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 0.5625;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.0625;
            }
            part def 'Mk1 Command Pod' :> 'Rocket Building'::'Pod';
            part def 'LV-T45 "Swivel" Liquid Fuel Engine' :> 'Rocket Building'::'Liquid Engine' {
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Liquid Engine'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Liquid Engine'::'Thrust' = 167.97;
                attribute 'Mass' : Real :>> 'Rocket Building'::'Kerbal Rocket Part'::Mass = 1.50;
            }
            part def 'RT-5 "Flea" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 1.50;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.45;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 140.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::'Thrust' = 162.91;
            }
            part def 'RT-10 "Hammer" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 3.56;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.75;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::Thrust = 197.90;
            }
        }
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import os
from pathlib import Path

import networkx as nx

import pymbe.api as pm
from pymbe.client import SysML2Client
from pymbe.label import get_label
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.calc_dependencies import generate_execution_order
from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *
from pymbe.query.metamodel_navigator import map_inputs_to_results

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

In [3]:
items = {
    "fts_full_mass": "Kerbal::'Rocket Building'::'Fuel Tank Section'::'Full Mass' [AttributeUsage]",  # Full Mass Attribute under Fuel Tank Sections
    "ft200_full_mass": "Kerbal::'Parts Library'::'FL-T200 Fuel Tank'::'Full Mass' [AttributeUsage]",  # Full Mass Attribute under FL-T200 Fuel Tank
    "ft100_full_mass": "Kerbal::'Parts Library'::'FL-T100 Fuel Tank'::'Full Mass' [AttributeUsage]",  # Full Mass Attribute under FL-T100 Fuel Tank
    "liquid_stage_full_mass": "Kerbal::'Rocket Building'::'Liquid Stage'::'Full Mass' [AttributeUsage]",  # Full Mass Attribute under Liquid Stage
    "top_plus": "Kerbal::'Rocket Building'::'Liquid Stage'::'Full Mass'::null [OperatorExpression]",  # The '+' Expression under the Full Mass Attribute under Liquid Stage
    "tank_mass_sum_1": "",  # The 'sum' Expression that sums Full Mass of tanks
}

In [4]:
collect_1 = 'd6644a0a-6eef-49c1-a770-60886073554c' # The 'collect' Expression that gathers Full Masses that apply to the current scope
collect_1_result = '2caccce7-a0b4-4926-8f24-0dbffb92f6ad' # The result parameter of above collect expression
full_mass_dot = 'ad0bff53-eebe-4446-a8df-4db0b7187707' # Expression that scopes Full Mass attribute in FeatureReferenceExpression
fre_1 = '2665fb1b-1f12-4f13-a977-0f060915773e' # Expression to point to the instances of Full Mass
fre_1_result = '6cfb516b-6045-454e-a521-83b747acef7e' # Result of gathering all instances of Full Mass from the interpretation
sum_1_result = '31f8c4bd-9700-4bc3-9970-3eb5451f0203' # Result of the sum Expression on Full Mass
x = '478a8a4d-90c7-44fd-a2ae-e4057636d4bc'

fre_2_result = '5897d247-1e81-41ad-bc5e-92eac8b35c2f' # Result of gathering all instances of Full Mass from the interpretation
collect_2_result = '2119c27e-d44c-479e-8b31-64de015dad61' # The result parameter of above collect expression
sum_2_collection = '3947bb94-0f09-413a-924b-2b1422354c93' # Input to sum Expression for Mass
sum_2_result = 'e50e47ab-4d6c-42d1-8997-81c52ffb7068' # Result of the sum Expression on Mass

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [5]:
helper_client = SysML2Client()

elements_data_path = Path("..") / "tests" / "fixtures" / "Kerbal.json"
assert elements_data_path.exists(), f"Could not find: '{elements_data_path}'"

helper_client._load_from_file(elements_data_path)

In [ ]:
id_map = {
    f"{data['qualifiedName']} [{data['@type']}]": id_
    for id_, data in helper_client.elements_by_id.items()
}
id_map

Create a graph representation of the model and load it into memory.

In [27]:
lpg = SysML2LabeledPropertyGraph()
lpg.elements_by_id = helper_client.elements_by_id

This is just a helper to make abbreviations more legible.

In [16]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [29]:
m0_interpretation = random_generator_playbook(
    lpg,
    shorten_pre_bake,
)

c:\sandboxes\pymbe\src\pymbe\graph\lpg.py:400: UserWarning: These edge types are not in the graph: {'ResultExpressionMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


KeyError: '6dec0903-3f71-4551-b3ff-74cc0da1ab33'

In [30]:
%debug

> c:\sandboxes\pymbe\src\pymbe\label.py(118)get_label_for_expression()
    116         path_step_names = []
    117         for owned_fm_id in expression["ownedFeatureMembership"]:
--> 118             owned_fm = all_elements[owned_fm_id["@id"]]
    119             if owned_fm["@type"] == "FeatureMembership":
    120                 member = all_elements[owned_fm["memberElement"]["@id"]]



ipdb>  owned_fm_id


{'@id': '6dec0903-3f71-4551-b3ff-74cc0da1ab33'}


ipdb>  all_elements


{'4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd': {'@type': 'FeatureReferenceExpression', '@id': '4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd', 'aliasId': [], 'behavior': [], 'documentation': [], 'documentationComment': [], 'effectiveName': None, 'endFeature': [], 'endOwningType': None, 'feature': [{'@id': '120e7fee-9442-4b72-ad37-48c3f42a6359'}, {'@id': '9743726d-901e-41c7-a6f9-4be933de5922'}], 'featureMembership': [{'@id': 'b93c584b-7d49-4f62-9f5a-9f01a7421810'}], 'featuringType': [{'@id': '86a423f7-18b6-4a86-ac52-2f69302121ab'}], 'function': None, 'humanId': None, 'identifier': '4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd', 'importedMembership': [], 'inheritedFeature': [], 'inheritedMembership': [], 'input': [], 'isAbstract': False, 'isComposite': False, 'isConjugated': False, 'isEnd': False, 'isModelLevelEvaluable': True, 'isNonunique': False, 'isOrdered': False, 'isSufficient': False, 'isUnique': True, 'member': [{'@id': '120e7fee-9442-4b72-ad37-48c3f42a6359'}, {'@id': '9743726d-901e-41c7-a6f9-4be933de5

ipdb>  l


    113     elif metatype == "InvocationExpression":
    114         prefix = type_names[0] if type_names else ""
    115     elif metatype == "PathStepExpression":
    116         path_step_names = []
    117         for owned_fm_id in expression["ownedFeatureMembership"]:
--> 118             owned_fm = all_elements[owned_fm_id["@id"]]
    119             if owned_fm["@type"] == "FeatureMembership":
    120                 member = all_elements[owned_fm["memberElement"]["@id"]]
    121                 # expect FRE here
    122                 if "referent" in member:
    123                     refered = all_elements[member["referent"]["@id"]]



ipdb>  c


To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [20]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

Once the interpretations are generated, we can look for expressions and create an execution order (this is similar to Excel builds a dependency graph internally to accelerate computations and partial updates when a user changes a cell value).

In [21]:
dcg = generate_execution_order(lpg, m0_interpretation)

One of the core tools in examining and working with the M1 model is using the get_projection function on the master graph to select out the kind of nodes and edges that will support other queries (roll-up using breadth-first search in reverse order, paths from one node to another to lay out sequences, etc.)

In [22]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred Graph"), m0_interpretation, dcg)

Display the calculation order determined by the algorithm, as well as a hint about what the step is (generating output from a function, moving values from function parameters to an attribute, applying redefinition, etc.)

In [28]:
lpg.nodes

{'4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd': {'@type': 'FeatureReferenceExpression',
  '@id': '4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd',
  'aliasId': [],
  'behavior': [],
  'documentation': [],
  'documentationComment': [],
  'effectiveName': None,
  'endFeature': [],
  'endOwningType': None,
  'feature': [{'@id': '120e7fee-9442-4b72-ad37-48c3f42a6359'},
   {'@id': '9743726d-901e-41c7-a6f9-4be933de5922'}],
  'featureMembership': [{'@id': 'b93c584b-7d49-4f62-9f5a-9f01a7421810'}],
  'featuringType': [{'@id': '86a423f7-18b6-4a86-ac52-2f69302121ab'}],
  'function': None,
  'humanId': None,
  'identifier': '4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd',
  'importedMembership': [],
  'inheritedFeature': [],
  'inheritedMembership': [],
  'input': [],
  'isAbstract': False,
  'isComposite': False,
  'isConjugated': False,
  'isEnd': False,
  'isModelLevelEvaluable': True,
  'isNonunique': False,
  'isOrdered': False,
  'isSufficient': False,
  'isUnique': True,
  'member': [{'@id': '120e7fee-9442-4b72-ad37

In [23]:
for item in dcg:
    instance_safe = True
    if item[0] not in m0_interpretation or len(m0_interpretation[item[0]]) == 0:
        print("No instances for " + lpg.nodes[item[0]]['qualifiedName'])
        instance_safe = False
    if item[1] not in m0_interpretation or len(m0_interpretation[item[1]]) == 0:
        print("No instances for " + lpg.nodes[item[1]]['qualifiedName'])
        instance_safe = False
    if instance_safe:
        rep_source = m0_interpretation[item[0]][0][-1]
        rep_target = m0_interpretation[item[1]][0][-1]
        if len(item) == 3:
            safe_item = item[2]
        else:
            safe_item = 'None'
        print('(' + str(rep_source) + ', ' + str(rep_target) + ', ' + safe_item + ')')

Use the calculation order in order to resolve the "unset" fields on many attributes to values where they are determined in the M1 model (e.g., using the ' = ' operator to assign values directly or connect to equations or analyses).

In [24]:
cg.solve_graph(lpg)

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

In [25]:
m0_interpretation[collect_1_result]

KeyError: '2caccce7-a0b4-4926-8f24-0dbffb92f6ad'

In [ ]:
m0_interpretation[collect_2_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#0 ([.Real#1966 (1.5), .Real#1435 (1.5), .Real#1293 (1.5), .Real#2380 (1.5)])],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#1 ([.Real#1285 (1.5)])],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#2 ([.Real#2666 (1.5), .Real#2614 (1.5), .Real#2611 (1.5)])]]

In [ ]:
m0_interpretation[sum_2_collection]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$collection#0 ([.Real#1966 (1.5), .Real#1435 (1.5), .Real#1293 (1.5), .Real#2380 (1.5)])],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.$collection#1 ([.Real#1285 (1.5)])],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.$collection#2 ([.Real#2666 (1.5), .Real#2614 (1.5), .Real#2611 (1.5)])]]

In [ ]:
m0_interpretation[sum_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$result#0 (10.125)],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.$result#1 (14.625)],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.$result#2 (1.125)]]

In [ ]:
m0_interpretation[x]

[[LS#0, + ($x, $y) => $result, LS#0.+ ($x, $y) => $result.$x#0 (6.0)],
 [LS#1, + ($x, $y) => $result, LS#1.+ ($x, $y) => $result.$x#1 (1.5)],
 [LS#2, + ($x, $y) => $result, LS#2.+ ($x, $y) => $result.$x#2 (4.5)]]

In [ ]:
m0_interpretation[liquid_stage_full_mass]

[[LS#0, .Real#1959 (16.125)],
 [LS#1, .Real#1532 (16.125)],
 [LS#2, .Real#106 (5.625)]]

In [ ]:
m0_interpretation[fre_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Full Mass (p) => $result,
  FRE.Full Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Full Mass (p) => $result.FRE.Full Mass.$result#0 ([[FL-T200#0, .Real#1135 (1.125)], [FL-T200#1, .Real#1051 (1.125)], [FL-T200#2, .Real#949 (1.125)], [FL-T200#3, .Real#137 (1.125)], [FL-T200#4, .Real#669 (1.125)], [FL-T200#5, .Real#2259 (1.125)], [FL-T200#6, .Real#1948 (1.125)], [FL-T200#7, .Real#1902 (1.125)], [FL-T200#8, .Real#856 (1.125)], [FL-T200#9, .Real#2071 (1.125)], [FL-T200#10, .Real#1014 (1.125)], [FL-T200#11, .Real#1751 (1.125)], [FL-T200#12, .Real#1871 (1.125)], [FL-T200#13, .Real#468 (1.125)], [FL-T200#14, .Real#176 (1.125)], [FL-T200#15, .Real#1388 (1.125)], [FL-T200#16, .Real#649 (1.125)], [FL-T200#17, .Real#380 (1.125)], [FL-T200#18, .Real#2595 (1.125)], [FL-T200#19, .Real#1292 (1.125)], [FL-T200#20, .Real#1445 (1.125)], [FL-T200

In [ ]:
m0_interpretation[fre_2_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Mass (p) => $result,
  FRE.Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Mass (p) => $result.FRE.Mass.$result#0 ([[Coupler#0, .Real#1836 (unset)], [Coupler#1, .Real#1601 (unset)], [Coupler#2, .Real#1363 (unset)], [Coupler#3, .Real#2123 (unset)], [Coupler#4, .Real#2456 (unset)], [Coupler#5, .Real#754 (unset)], [Coupler#6, .Real#764 (unset)], [Coupler#7, .Real#1574 (unset)], [Coupler#8, .Real#1876 (unset)], [Coupler#9, .Real#1604 (unset)], [Coupler#10, .Real#1719 (unset)], [Coupler#11, .Real#104 (unset)], [Coupler#12, .Real#726 (unset)], [Coupler#13, .Real#722 (unset)], [Coupler#14, .Real#1414 (unset)], [Coupler#15, .Real#661 (unset)], [Coupler#16, .Real#81 (unset)], [Coupler#17, .Real#2119 (unset)], [Coupler#18, .Real#805 (unset)], [Coupler#19, .Real#2025 (unset)], [Coupler#20, .Real#1673 (unset)], [Coupler#21, .Real#2050 (unse

Show all interpretation sequence sets (limited to length of 5).

In [11]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)

KeyError: 'e310d81d-8b19-4407-a090-b3272a8f3a49'

In [12]:
%debug

> c:\sandboxes\pymbe\src\pymbe\interpretation\results.py(8)pprint_interpretation()
      6     print_lines = []
      7     for key, val in interpretation.items():
----> 8         print_lines.append(get_label(all_elements[key], all_elements) + ', id = ' + key + ', size = ' + str(len(val)))
      9         short_list = []
     10         for indx, ind_val in enumerate(val):



ipdb>  key


'e310d81d-8b19-4407-a090-b3272a8f3a49'


ipdb>  all_elements


{'2759f68a-b09e-44ff-9b45-c4ba28314ecc': {}, '19a5cf60-f6c1-4cba-9f9d-0b68e935de61': {}, 'fbfa52d6-8b81-4087-b908-11851f55eb11': {}, '3923d9f4-e30e-4b56-8862-0b7463a53234': {}, '6b19ebe1-6d95-4b72-ba16-247e6733b56c': {}, 'd2ccb1d4-015f-4d74-82ee-b795c2ea1c16': {}, 'dcc52f93-c89b-4b62-a94e-c50035648450': {}, '6d907fe9-d83e-4c7e-833b-647f9c1aebe3': {}, '0cf78450-b602-4e1c-b749-55d50a94d0f0': {}, 'f7d4ac57-9a58-4bec-9153-2254019de47d': {}, '4eb53d3c-c93d-4900-9885-6ec99150dc01': {}, 'e5d649cf-b6cf-4426-b778-516a2d900029': {}, '793f6dd7-c561-472a-8d50-ea4d0a6f4b8f': {}, '813c51ee-98be-4256-8f2c-3c174e249af5': {}, 'aef934b4-e1e9-446e-9ece-90cbec19d4b0': {}, '8ec3900c-d93c-41fd-9ce2-f13e80fec76c': {}, 'd79192d7-bf71-49c8-b671-dab115275f7d': {}, '699c2a6c-ef62-4ccb-b6fa-9815e9eba978': {}, 'f8c8d8e8-f215-43c6-9fb4-5738ba0f062e': {}, 'e7658828-6e2a-4041-9d1d-a599115b3859': {}, '120e7fee-9442-4b72-ad37-48c3f42a6359': {}, '4fb2f5a0-f6a4-4cfe-8a58-a66b7498effd': {}, 'a3ce36a4-de64-45b8-9197-d5b77b

ipdb>  ll


      5 def pprint_interpretation(interpretation: dict, all_elements: dict) -> list:
      6     print_lines = []
      7     for key, val in interpretation.items():
----> 8         print_lines.append(get_label(all_elements[key], all_elements) + ', id = ' + key + ', size = ' + str(len(val)))
      9         short_list = []
     10         for indx, ind_val in enumerate(val):
     11             if indx < 5:
     12                 short_list.append(ind_val)
     13         if len(val) > 4:
     14             short_list.append(['..'])
     15 
     16         print_lines.extend(short_list)
     17     return print_lines
     18 



ipdb>  u


> <ipython-input-11-d3599438ddda>(1)<module>()
----> 1 for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
      2     print(print_line)



ipdb>  c
